**Using the Kaggle API to downlaod the necessary dataset**

In [22]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files("blastchar/telco-customer-churn", path=".", unzip=True)


Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn


**Addition commands using the kaggle API**

In [23]:
# Extracts a JSON file of the metadata for a given dataset
kaggle.api.dataset_metadata("blastchar/telco-customer-churn", path=".")

# List all the data files in a given dataset
print(kaggle.api.dataset_list_files("blastchar/telco-customer-churn").files)

# Provides a list of dataset that contain the search word(s)
datasets = kaggle.api.dataset_list(search="customer churn")
print(datasets)

[]
[blastchar/telco-customer-churn, barun2104/telecom-churn, radheshyamkollipara/bank-customer-churn, muhammadshahidazeem/customer-churn-dataset, gauravtopre/bank-customer-churn-dataset, shubhammeshram579/bank-customer-churn-prediction, shantanudhakadd/bank-customer-churn-prediction, anandshaw2001/customer-churn-dataset, rashadrmammadov/customer-churn-dataset, saurabhbadole/bank-customer-churn-prediction-dataset, mathchi/churn-for-bank-customers, abdullah0a/telecom-customer-churn-insights-for-analysis, santoshd3/bank-customers, divu2001/customer-churn-rate, abdallahwagih/telco-customer-churn, alfathterry/telco-customer-churn-11-1-3, mehmetsabrikunt/internet-service-churn, shilongzhuang/telecom-customer-churn-by-maven-analytics, tejashvi14/tour-travels-customer-churn-prediction, yeanzc/telco-customer-churn-ibm-dataset]


**Assign the dataset to the "data" variable so we can analyse it**

In [22]:
import pandas as pd

data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

**Obtain information about the data set, each column (such as data types)** 

In [23]:
# To view all column names and their respective data types
data.columns
data.info()
data.describe() # Shows statistical summaries for all numeric columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


**Perform data cleaning**

In [29]:
# get the number of missing data points per column
missing_values_count = data.isnull().sum()
missing_values_count

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

It is clear to see that there are no missing values in this datset

In [30]:
unique_values = data.nunique(axis=0)
unique_values


customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

**TotalCharges might be a string in some cases. Therefore, we need to convert it to numeric before training:**

In [31]:
import pandas as pd

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')